In [ ]:
# Importing libraries

import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, accuracy_score

In [22]:
# Loading data

df = pd.read_csv('data/dataset.csv')
df = df.dropna()
df

,id,local_id,link,title,content,gold_label
0,1,1,https://urdu.arynews.tv/car-sales-in-pakistan/,پاکستان میں گاڑیوں کی فروخت میں بڑا اضافہ,ملکی آٹو سیکٹر سے زبردست خبر آگئی۔ پاکستان می...,Business
1,2,2,https://urdu.arynews.tv/gold-rates-in-pakistan-3/,پاکستان میں سونے کی قیمت آج کتنی کم ہوئی؟,کراچی: کاروباری ہفتے کے پہلے روز سونے کی قیمت ...,Business
2,3,5,https://urdu.arynews.tv/cotton-production-cott...,امریکا سے معیاری روئی کی درآمد بڑھ گئی,کراچی: پاکستان میں کپاس کی پیداوار میں کمی کے ...,Business
3,4,3,https://urdu.arynews.tv/psx-today-11-nov/,پاکستان اسٹاک ایکسچینج میں نئی تاریخ رقم,پاکستان اسٹاک ایکسچینج نے ایک اور سنگ میل عبور...,Business
4,5,4,https://urdu.arynews.tv/ghee-and-cooking-oil-p...,عوام کے لیے نئی مشکل : گھی اور کوکنگ آئل کی قی...,لاہور : گھی اور کوکنگ آئل کی قیمتوں میں ایک با...,Business
...,...,...,...,...,...,...
5938,5939,492,https://jang.com.pk/news/1015053,کراچی یونیورسٹی نے ہاکی فائنل جیت لیا,ہائرایجوکیشن کمیشن (ایچ ای سی، زون جی ) ہاکی چ...,Sports
5939,5940,495,https://jang.com.pk/news/1012389,قوم کو ایک بار پھر کرکٹ نے متحد کردیا,کئی سالوں کے بعد پہلی بار ایسا ہوا ہے کہ پاکست...,Sports
5940,5941,496,https://jang.com.pk/news/1012388,جامعہ این ای ڈی میں فلڈ لائٹ کرکٹ گراؤنڈ کا اف...,رواں ماہ ہر جانب ورلڈ کپ کی گہما گہمی جاری ہے،...,Sports
5941,5942,498,https://jang.com.pk/news/1012386,اسپورٹس مقابلے,الفا ایجوکیشن نیٹ ورک پر جہاں نصابی سرگرمیاں ب...,Sports


In [23]:
# Add this function to perform stemming
def simple_urdu_stemmer(word):
    # Define common suffixes in Urdu
    suffixes = ['یں', 'اں', 'وں', 'یں', 'ہاں', 'ی', 'ے', 'و', 'ہ']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

# Loading Urdu stopwords from the json file
with open('data/kaggle_stopwords.json', 'r', encoding='utf-8') as file:
    urdu_stopwords = set(json.load(file).keys())

#Loading Shanzae Stopwords
with open('data/shanzae/stopwords.json', 'r', encoding='utf-8') as file:
    shanzae_stopwords = set(json.load(file).keys())

#Loading Yamsheen Stopwords
with open('data/yamsheen/stopwords.json', 'r', encoding='utf-8') as file:
    yamsheen_stopwords = set(json.load(file).keys())

# Function to clean our Urdu sentences
def clean_content(text, stopwords):
    # Remove punctuation and numbers
    text = str(text)
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords)
    text = ' '.join(word for word in text.split() if word not in shanzae_stopwords)
    text = ' '.join(word for word in text.split() if word not in yamsheen_stopwords)
    # Convert to lowercase
    text = text.lower()
    #Use the simple_urdu_stemmer
    text = ' '.join(simple_urdu_stemmer(word) for word in text.split())
    return text

df['content'] = df['content'].apply(lambda x: clean_content(x, urdu_stopwords))

df.head()

,id,local_id,link,title,content,gold_label
0,1,1,https://urdu.arynews.tv/car-sales-in-pakistan/,پاکستان میں گاڑیوں کی فروخت میں بڑا اضافہ,ملک آٹ سیکٹر س زبردست خبر آگئی۔ پاکستان گاڑی ...,Business
1,2,2,https://urdu.arynews.tv/gold-rates-in-pakistan-3/,پاکستان میں سونے کی قیمت آج کتنی کم ہوئی؟,کراچ کاروبار ہفت پہل روز سون قیمت رجحان رہا۔ پ...,Business
2,3,5,https://urdu.arynews.tv/cotton-production-cott...,امریکا سے معیاری روئی کی درآمد بڑھ گئی,کراچ پاکستان کپاس پیداوار باعث اسپننگ مل س معی...,Business
3,4,3,https://urdu.arynews.tv/psx-today-11-nov/,پاکستان اسٹاک ایکسچینج میں نئی تاریخ رقم,پاکستان اسٹاک ایکسچینج ن میل عبور لیا۔ کاروبار...,Business
4,5,4,https://urdu.arynews.tv/ghee-and-cooking-oil-p...,عوام کے لیے نئی مشکل : گھی اور کوکنگ آئل کی قی...,لاہور کوکنگ آئل قیمت اضاف ہوا، قمیت س تجاوز کر...,Business


Neural Network

In [28]:
# 1. Prepare the data
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])

# Prepare labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform([[label] for label in df['gold_label']])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to dense arrays
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

class MultiLabelNeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01):
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.layers_sizes = [input_size] + hidden_sizes + [output_size]
        self.weights = []
        self.biases = []
        
        for i in range(len(self.layers_sizes) - 1):
            self.weights.append(np.random.randn(self.layers_sizes[i], self.layers_sizes[i+1]) * np.sqrt(2./self.layers_sizes[i]))
            self.biases.append(np.zeros((1, self.layers_sizes[i+1])))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward_propagation(self, X):
        self.activations = [X]
        for i in range(len(self.weights)):
            net = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.activations.append(self.sigmoid(net))
        return self.activations[-1]

    def backward_propagation(self, X, y, output):
        m = X.shape[0]
        delta = output - y
        
        dWeights = []
        dBiases = []
        
        for i in range(len(self.weights) - 1, -1, -1):
            dW = np.dot(self.activations[i].T, delta) / m
            db = np.sum(delta, axis=0, keepdims=True) / m
            
            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_derivative(self.activations[i])
            
            dWeights.insert(0, dW)
            dBiases.insert(0, db)
        
        return dWeights, dBiases

    def fit(self, X, y, epochs=100, batch_size=32, verbose=True):
        m = X.shape[0]
        
        for epoch in range(epochs):
            # Mini-batch gradient descent
            indices = np.random.permutation(m)
            X = X[indices]
            y = y[indices]
            
            for i in range(0, m, batch_size):
                batch_X = X[i:i+batch_size]
                batch_y = y[i:i+batch_size]
                
                output = self.forward_propagation(batch_X)
                dWeights, dBiases = self.backward_propagation(batch_X, batch_y, output)
                
                for j in range(len(self.weights)):
                    self.weights[j] -= self.learning_rate * dWeights[j]
                    self.biases[j] -= self.learning_rate * dBiases[j]
            
            if verbose and epoch % 10 == 0:
                predictions = self.predict(X)
                accuracy = np.mean(np.all(predictions == y, axis=1))
                print(f"Epoch {epoch}, Accuracy: {accuracy:.4f}")

    def predict(self, X):
        output = self.forward_propagation(X)
        return (output >= 0.5).astype(int)

# Initialize and train the network
input_size = X_train_dense.shape[1]
hidden_sizes = [256, 128]
output_size = y_train.shape[1]

nn = MultiLabelNeuralNetwork(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
    learning_rate=0.01
)

# Train
nn.fit(X_train_dense, y_train, epochs=500, batch_size=32)

# Evaluate
y_pred = nn.predict(X_test_dense)
print("\nNeural Network Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

Epoch 0, Accuracy: 0.0000
Epoch 10, Accuracy: 0.4713
Epoch 20, Accuracy: 0.7571
Epoch 30, Accuracy: 0.8440
Epoch 40, Accuracy: 0.8744
Epoch 50, Accuracy: 0.8889
Epoch 60, Accuracy: 0.9015
Epoch 70, Accuracy: 0.9105
Epoch 80, Accuracy: 0.9178
Epoch 90, Accuracy: 0.9259
Epoch 100, Accuracy: 0.9313
Epoch 110, Accuracy: 0.9351
Epoch 120, Accuracy: 0.9399
Epoch 130, Accuracy: 0.9435
Epoch 140, Accuracy: 0.9486
Epoch 150, Accuracy: 0.9510
Epoch 160, Accuracy: 0.9542
Epoch 170, Accuracy: 0.9580
Epoch 180, Accuracy: 0.9595
Epoch 190, Accuracy: 0.9617
Epoch 200, Accuracy: 0.9647
Epoch 210, Accuracy: 0.9645
Epoch 220, Accuracy: 0.9658
Epoch 230, Accuracy: 0.9673
Epoch 240, Accuracy: 0.9694
Epoch 250, Accuracy: 0.9705
Epoch 260, Accuracy: 0.9730
Epoch 270, Accuracy: 0.9726
Epoch 280, Accuracy: 0.9747
Epoch 290, Accuracy: 0.9767
Epoch 300, Accuracy: 0.9795
Epoch 310, Accuracy: 0.9797
Epoch 320, Accuracy: 0.9801
Epoch 330, Accuracy: 0.9807
Epoch 340, Accuracy: 0.9831
Epoch 350, Accuracy: 0.9844
Epo

C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Testing

In [29]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# Load the new dataset
new_df = pd.read_csv('./data/bbc_dataset.csv')

# Preprocess the content
new_df['content'] = new_df['content'].apply(lambda x: clean_content(x, urdu_stopwords))

# Transform the new data using the existing vectorizer
X_new = vectorizer.transform(new_df['content'])

# Convert to dense array
X_new_dense = X_new.toarray()

# Predict using the trained model
y_pred_new = nn.predict(X_new_dense)

# If the new dataset has labels, evaluate the performance
if 'gold_label' in new_df.columns:
    # Prepare labels
    new_df['gold_label'] = new_df['gold_label'].fillna('')
    new_df['gold_label'] = new_df['gold_label'].str.split(',')
    new_df['gold_label'] = new_df['gold_label'].apply(lambda x: [label.strip() for label in x if label.strip()])
    y_new = mlb.transform(new_df['gold_label'])

    # Evaluate performance
    print("Accuracy on new dataset:", accuracy_score(y_new, y_pred_new))
    print("\nClassification Report on new dataset:")
    print(classification_report(y_new, y_pred_new, target_names=mlb.classes_))
else:
    # Convert predictions to labels
    predicted_labels = mlb.inverse_transform(y_pred_new)

    # Add predictions to the dataframe
    new_df['predicted_labels'] = [';'.join(labels) for labels in predicted_labels]

    # Save predictions to CSV
    new_df.to_csv('./data/dawn_dataset_predictions.csv', index=False)

    # Display the predictions
    print(new_df[['content', 'predicted_labels']])

Accuracy on new dataset: 0.5352591333899746

Classification Report on new dataset:
                    precision    recall  f1-score   support

          Business       0.76      0.24      0.37       223
     Entertainment       0.51      0.87      0.65       240
     International       0.66      0.56      0.61       234
Science-Technology       0.48      0.29      0.36       240
            Sports       0.95      0.83      0.89       240

         micro avg       0.64      0.56      0.60      1177
         macro avg       0.67      0.56      0.57      1177
      weighted avg       0.67      0.56      0.58      1177
       samples avg       0.55      0.56      0.55      1177



C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# 1. Prepare the data
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])

# Prepare labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform([[label] for label in df['gold_label']])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to dense arrays
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

class MultiLabelNeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01):
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.layers_sizes = [input_size] + hidden_sizes + [output_size]
        self.weights = []
        self.biases = []
        
        for i in range(len(self.layers_sizes) - 1):
            self.weights.append(np.random.randn(self.layers_sizes[i], self.layers_sizes[i+1]) * np.sqrt(2./self.layers_sizes[i]))
            self.biases.append(np.zeros((1, self.layers_sizes[i+1])))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward_propagation(self, X):
        self.activations = [X]
        for i in range(len(self.weights)):
            net = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.activations.append(self.sigmoid(net))
        return self.activations[-1]

    def backward_propagation(self, X, y, output):
        m = X.shape[0]
        delta = output - y
        
        dWeights = []
        dBiases = []
        
        for i in range(len(self.weights) - 1, -1, -1):
            dW = np.dot(self.activations[i].T, delta) / m
            db = np.sum(delta, axis=0, keepdims=True) / m
            
            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_derivative(self.activations[i])
            
            dWeights.insert(0, dW)
            dBiases.insert(0, db)
        
        return dWeights, dBiases

    def fit(self, X, y, epochs=100, batch_size=32, verbose=True):
        m = X.shape[0]
        
        for epoch in range(epochs):
            # Mini-batch gradient descent
            indices = np.random.permutation(m)
            X = X[indices]
            y = y[indices]
            
            for i in range(0, m, batch_size):
                batch_X = X[i:i+batch_size]
                batch_y = y[i:i+batch_size]
                
                output = self.forward_propagation(batch_X)
                dWeights, dBiases = self.backward_propagation(batch_X, batch_y, output)
                
                for j in range(len(self.weights)):
                    self.weights[j] -= self.learning_rate * dWeights[j]
                    self.biases[j] -= self.learning_rate * dBiases[j]
            
            if verbose and epoch % 10 == 0:
                predictions = self.predict(X)
                accuracy = np.mean(np.all(predictions == y, axis=1))
                print(f"Epoch {epoch}, Accuracy: {accuracy:.4f}")

    def predict(self, X):
        output = self.forward_propagation(X)
        return (output >= 0.5).astype(int)

# Initialize and train the network
input_size = X_train_dense.shape[1]
hidden_sizes = [256, 128]
output_size = y_train.shape[1]

nn = MultiLabelNeuralNetwork(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
    learning_rate=0.01
)

# Train
nn.fit(X_train_dense, y_train, epochs=500, batch_size=32)

# Evaluate
y_pred = nn.predict(X_test_dense)
print("\nNeural Network Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

Epoch 0, Accuracy: 0.0000
Epoch 10, Accuracy: 0.4713
Epoch 20, Accuracy: 0.7571
Epoch 30, Accuracy: 0.8440
Epoch 40, Accuracy: 0.8744
Epoch 50, Accuracy: 0.8889
Epoch 60, Accuracy: 0.9015
Epoch 70, Accuracy: 0.9105
Epoch 80, Accuracy: 0.9178
Epoch 90, Accuracy: 0.9259
Epoch 100, Accuracy: 0.9313
Epoch 110, Accuracy: 0.9351
Epoch 120, Accuracy: 0.9399
Epoch 130, Accuracy: 0.9435
Epoch 140, Accuracy: 0.9486
Epoch 150, Accuracy: 0.9510
Epoch 160, Accuracy: 0.9542
Epoch 170, Accuracy: 0.9580
Epoch 180, Accuracy: 0.9595
Epoch 190, Accuracy: 0.9617
Epoch 200, Accuracy: 0.9647
Epoch 210, Accuracy: 0.9645
Epoch 220, Accuracy: 0.9658
Epoch 230, Accuracy: 0.9673
Epoch 240, Accuracy: 0.9694
Epoch 250, Accuracy: 0.9705
Epoch 260, Accuracy: 0.9730
Epoch 270, Accuracy: 0.9726
Epoch 280, Accuracy: 0.9747
Epoch 290, Accuracy: 0.9767
Epoch 300, Accuracy: 0.9795
Epoch 310, Accuracy: 0.9797
Epoch 320, Accuracy: 0.9801
Epoch 330, Accuracy: 0.9807
Epoch 340, Accuracy: 0.9831
Epoch 350, Accuracy: 0.9844
Epo

C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
